In [1]:
import anthropic
import os
import base64
import glob
import time

from dotenv import load_dotenv
from PIL import Image
from io import BytesIO
from tqdm.auto import tqdm

In [2]:
load_dotenv()

True

In [3]:
client = anthropic.Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))

In [4]:
model = 'claude-3-5-haiku-latest'

In [5]:
image_paths_list = [
    '../../input/sroie_v2/SROIE2019/train/img/*.jpg', # Train.
    '../../input/sroie_v2/SROIE2019/test/img/*.jpg' # Test
] 

In [6]:
out_dir_list = [
    '../../input/claude35haiku_annots/train_annots', #Train
    '../../input/claude35haiku_annots/test_annots' # Test
]

In [7]:
prompt = """Give the OCR text from this image and nothing else."""

In [8]:
for image_path_split, out_dir in zip(image_paths_list, out_dir_list):
    all_images = glob.glob(image_path_split)
    os.makedirs(out_dir, exist_ok=True)

    for image_path in tqdm(all_images, total=len(all_images)):
        image = Image.open(image_path)
        image = image.resize((512, 768))
        
        buffered = BytesIO()
        image.save(buffered, format="JPEG")
        img_data = base64.b64encode(buffered.getvalue()).decode('utf-8')
        
        message = client.messages.create(
            model='claude-3-5-haiku-latest',
            max_tokens=1024,
            messages=[
                {
                    'role': 'user',
                    'content': [
                        {
                            'type': 'image',
                            'source': {
                                'type': 'base64',
                                'media_type': 'image/jpeg',
                                'data': img_data,
                            },
                        },
                        {
                            'type': 'text',
                            'text': prompt
                        }
                    ],
                }
            ],
        )

        with open(os.path.join(out_dir, image_path.split(os.path.sep)[-1].split('.jpg')[0]+'.txt'), 'w') as f:
            f.write(message.content[0].text)
    
        time.sleep(3)

  0%|          | 0/626 [00:00<?, ?it/s]

  0%|          | 0/347 [00:00<?, ?it/s]

In [9]:
# print(message.content[0].text)